# 量子デバイス（QPU）の特性の違いを理解する
それぞれのデバイスでは、使えるゲートが異なります。Amazon Braket では前述の `braket.circuits` クラスで回路とゲート操作が定義されており、`Circuit()` で作成した回路に対してゲート操作ができます。Simulator は全てのゲートを利用可能ですが、QPU によってはサポートされているゲートが異なります。

## シミュレータで利用できる回路演算について
SV1 などのシミュレータでは、[Amazon Braket SDK で実装されているゲートが全て利用可能](https://docs.aws.amazon.com/braket/latest/developerguide/braket-constructing-circuit.html)です。サポートしている演算（ユニタリーオペレータ）を表示します

In [1]:
from braket.circuits import Gate
from braket.aws import AwsDevice
import string

# print all (the usual suspects) available gates currently available within SDK
gate_set = [attr for attr in dir(Gate) if attr[0] in string.ascii_uppercase]
print('Gate set supported by SDK:\n', gate_set)
print('\n') 

Gate set supported by SDK:
 ['CCNot', 'CNot', 'CPhaseShift', 'CPhaseShift00', 'CPhaseShift01', 'CPhaseShift10', 'CSwap', 'CV', 'CY', 'CZ', 'ECR', 'GPi', 'GPi2', 'H', 'I', 'ISwap', 'MS', 'PSwap', 'PhaseShift', 'Rx', 'Ry', 'Rz', 'S', 'Si', 'Swap', 'T', 'Ti', 'Unitary', 'V', 'Vi', 'X', 'XX', 'XY', 'Y', 'YY', 'Z', 'ZZ']




---
## IonQ について
IonQ は、イオントラップという原理で作られた11量子ビットで構成されるデバイスです。

デバイスタイプとして、Rigetti の ARN `arn:aws:braket:::device/qpu/ionq/ionQdevice` を指定し、このデバイスで利用できる量子ゲートのタイプと、量子ビット間の結合（トポロジー）について調べてみます。

In [2]:
# import the device module
from braket.aws import AwsDevice

device = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
supported_gates = device.properties.action['braket.ir.jaqcd.program'].supportedOperations
# print the supported gate set
print('Gate set supported by the IonQ device:\n', supported_gates)

Gate set supported by the IonQ device:
 ['x', 'y', 'z', 'rx', 'ry', 'rz', 'h', 'cnot', 's', 'si', 't', 'ti', 'v', 'vi', 'xx', 'yy', 'zz', 'swap', 'i']


In [3]:
# take a look at the device connectivity graph
device.properties.dict()['paradigm']['connectivity']

{'fullyConnected': True, 'connectivityGraph': {}}

IonQ は、11量子ビットがのそれぞれの量子ビットがそれ以外の全ての量子ビットと結合している、いわゆる全結合のトポロジーとなっています。そのため、任意の２量子ビットに対して、それぞれのデバイスに対してゲート操作が可能です。

---
## Rigetti について
Rigetti Aspen M-2 は、超伝導量子ビットで構成された80量子ビットのデバイスです。

デバイスタイプとして、Rigetti の ARN `arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-1` を指定します。

In [6]:
device = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-2")
supported_gates = device.properties.action['braket.ir.jaqcd.program'].supportedOperations
# print the supported gate set
print('Gate set supported by the Rigetti device:\n', supported_gates)

Gate set supported by the Rigetti device:
 ['cz', 'xy', 'ccnot', 'cnot', 'cphaseshift', 'cphaseshift00', 'cphaseshift01', 'cphaseshift10', 'cswap', 'h', 'i', 'iswap', 'phaseshift', 'pswap', 'rx', 'ry', 'rz', 's', 'si', 'swap', 't', 'ti', 'x', 'y', 'z', 'start_verbatim_box', 'end_verbatim_box']


In [7]:
# take a look at the device connectivity graph
device.properties.dict()['paradigm']['connectivity']

{'fullyConnected': False,
 'connectivityGraph': {'0': ['1', '7'],
  '1': ['0', '16', '2'],
  '10': ['11', '113', '17'],
  '100': ['101', '107'],
  '101': ['100', '102', '116'],
  '102': ['101', '103', '115'],
  '103': ['102', '104'],
  '104': ['103', '105', '7'],
  '105': ['104', '106'],
  '106': ['105', '107'],
  '107': ['100', '106'],
  '11': ['10', '12', '26'],
  '110': ['111', '117'],
  '111': ['110', '112', '126'],
  '112': ['111', '113'],
  '113': ['10', '112', '114'],
  '114': ['113', '115', '17'],
  '115': ['102', '114', '116'],
  '116': ['101', '115', '117'],
  '117': ['110', '116'],
  '12': ['11', '13', '25'],
  '120': ['121', '127'],
  '121': ['120', '122', '136'],
  '122': ['121', '123', '135'],
  '123': ['122', '124', '20'],
  '124': ['123', '125', '27'],
  '125': ['124', '126'],
  '126': ['111', '125', '127'],
  '127': ['120', '126'],
  '13': ['12', '14'],
  '130': ['131', '137'],
  '131': ['130', '132', '146'],
  '132': ['131', '133', '145'],
  '133': ['132', '134', '30'

上記のように、Rigetti は 量子ビットがそれぞれ、２量子ビット、または３量子ビットに対して結合していることがわかります。この結合規則を見ながら、操作可能な量子ゲートをプログラミングして量子計算を実施する必要があります。

----
## Oxford Quantum Circuits (OQC) について
OQC の Lucy は超伝導量子ビットで構成された8量子ビットのデバイスです。

ARN `arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy` を指定します。

In [ ]:
device = AwsDevice("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")
supported_gates = device.properties.action['braket.ir.jaqcd.program'].supportedOperations
# print the supported gate set
print('Gate set supported by the OQC device:\n', supported_gates)

---

## D-Waveについて

量子アニーラは、組合せ最適化問題、特に2次制約なし最適化（QUBO）のクラスに属する問題を解決するために設計された特殊用途の量子コンピューターです。 Amazon Braket を使用すると、Braket-Ocean プラグインを介して D-Wave の Ocean ソフトウェアを使用して D-Wave QPU をネイティブにプログラムできます。Amazon Braket ノートブックインスタンスには、Ocean と Braket-Ocean プラグインがプリインストールされています。

D-Wave は複数のデバイスが利用可能です。

### D-Wave 2000Q
ARN `arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6` を指定します。

In [6]:
device = AwsDevice('arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6')

# take a look at the device connectivity graph
topology = device.properties.dict()['provider']['topology']
qubit_number = device.properties.dict()['provider']['qubitCount']
print(topology)
print('qubit number = ', qubit_number)

{'type': 'chimera', 'shape': [16, 16, 4]}
qubit number =  2048


### D-Wave Advantage 4.1
ARN `arn:aws:braket:::device/qpu/d-wave/Advantage_system4` を指定します。

In [7]:
device = AwsDevice('arn:aws:braket:::device/qpu/d-wave/Advantage_system4')

topology = device.properties.dict()['provider']['topology']
qubit_number = device.properties.dict()['provider']['qubitCount']
print(topology)
print('qubit number = ', qubit_number)

{'type': 'pegasus', 'shape': [16]}
qubit number =  5760


---

これで、QPU のそれぞれの特性を知ることができました。それぞれの QPU のデバイス特性を意識しながら、量子計算のコードを書いていきましょう。